# IMPORT LIBRARIES

In [1]:
# WIDGETS
from ipywidgets import interact
import ipywidgets as widgets

# DATA
from ecmwf.opendata import Client
from eccodes import *
import numpy as np
import xarray as xr

# manage data
import pdbufr # manage bufr data
from datetime import datetime, timedelta # date functions
import pandas as pd # manage dataframes
from Magics.macro import * # load grib data

# plot mapa
import ipyleaflet

from ipyleaflet import Map, Heatmap, LayersControl, LayerGroup

# PARAMETERS (will be read from main widget)

In [2]:
time = 0
#date = 20230806
value_date = 20220925 # ha de ser entre avui i 5 dies enrere
step = 12

initial_lat_lon = (38, -90)

#########################################
#    date=0, # today
#    date=-1, # yesterday
#    date=-2, # the day before yesterday

# other parameters

In [3]:
outfolder = ''


# variable initialization

In [4]:
lat = initial_lat_lon[0]
lon = initial_lat_lon[1]
atm_variables = ['2t', 'mslp']


# DOWNLOAD DATA

In [14]:
# https://github.com/ecmwf/ecmwf-opendata
#for var in atm_variables:
#    print(var)
#    filename = outfolder + var + '_' + str(value_date) + '.grib'
#    client = Client("https://data.ecmwf.int/forecasts", beta=True) # ecwf: last five days
#    client.retrieve(
#        date = value_date, #date start of the forecast
#        time = time,  # time start of the forecast or 12
#        step = 240, #step of the forecast
#        stream = "oper",
#        type = "fc",
#        levtype = "sfc",
#        param = var,
#        target = filename
#)

# LOAD DATA 

In [5]:
print('Downloading data...')
for var in atm_variables:
   # print(var)
    filename = outfolder + var + '_' + str(value_date) + '.grib'
    
    # initialize locations vector
    locations = []
    
    # load data
    with xr.open_dataset(filename, engine="cfgrib") as data:
        ds = data
        
    # extract matrix values from data
    if var == '2t':
       # var_latlon = ds.t2m.values
        var_latlon = ds.sst.values[0,:,:]

    elif var == 'msl':
        var_latlon = ds.msl.values[0,:,:]
    
    # save lat lon value into locations vector:
    for ii in range(0,len(ds.latitude),1):
        for jj in range(0,len(ds.longitude),1):
            locs = [ds.latitude.values[ii], ds.longitude.values[jj], var_latlon[ii,jj]]
            locations.append(locs)
            
    if var == '2t':
        locations_2t = locations
    elif var == 'mslp':
        locations_msl = locations


print('done!')

2t
mslp
done!


# ATMOSPHERIC PLOTS

In [37]:
# create empty map
map_atm_var = ipyleaflet.Map(
    center= initial_lat_lon,
    basemap=ipyleaflet.basemaps.OpenStreetMap.France,
    zoom = 3.5)

# widget to select atmospheric variable
sel_atm_var = widgets.Dropdown(
    options = atm_variables,
    description = 'Atmospheric variable:',
    disabled=False,
    value = 'mslp',
)

if sel_atm_var.value == '2t':
    locations = locations_2t
elif sel_atm_var.value == 'mslp':
    locations = locations_msl

def update_atm_layer(change):
  #  map_atm_var.remove_layer(atm_layer)
    if change.new == '2t':
     #locations = locations_2t
       atm.locations = locations_2t
    
    elif change.new == 'mslp':
        atm_layer.locations = locations_msl
        #locations = locations_msl
   # map_atm_var.add_layer(atm_layer)
    

atm_layer = Heatmap(locations=locations, radius=5)
    
sel_atm_var.observe(update_atm_layer, names = 'value')

map_atm_var.add_layer(atm_layer)
map_atm_var.add_control(LayersControl());
#display(sel_atm_var)
#display(map_atm_var)

Dropdown(description='Atmospheric variable:', index=1, options=('2t', 'mslp'), value='mslp')

Map(center=[38, -90], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

NameError: name 'atm' is not defined

In [38]:
map2 = map_atm_var

In [47]:
grid = widgets.GridspecLayout(2,2, layout=widgets.Layout(justify_content = 'center'))
grid[0,0] = sel_atm_var
grid[1,0] = map_atm_var
grid[1,1] = map2

In [48]:
display(grid)

GridspecLayout(children=(Dropdown(description='Atmospheric variable:', layout=Layout(grid_area='widget001'), o…

In [50]:
widgets.VBox([sel_atm_var])
map_atm_var

Map(bottom=1780.0, center=[38, -90], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…